In [174]:
import geopy
import pandas as pd
import numpy as np

from pyproj import Geod
from shapely.geometry import Point, LineString

X = pd.read_csv('data/stores_train.csv')
spatial = pd.read_csv('data/grunnkrets_norway_stripped.csv')


In [175]:
# df = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
#                               'Parrot', 'Parrot'],
#                    'Max Speed': [380., 370., 24., 26.]})
# df.groupby("Animal").apply(lambda grp: -grp.count() / df.shape[0])

gk_muni = spatial[['grunnkrets_id', 'municipality_name']]

X = X.merge(gk_muni, on='grunnkrets_id')
X = X.merge(X.groupby(['municipality_name'])['lat', 'lon', 'revenue'].apply(lambda x: x.sum() / (x.count()))[['lat', 'lon']], on='municipality_name', suffixes=(None, '_center'))
# X = X.groupby(['municipality_name'])['lat', 'lon', 'revenue'].apply(lambda x: (x[['lat', 'lon']] * x['revenue']).sum() / (x.count())).to_frame('ballefaen')

def meter_distance(lat1, lon1, lat2, lon2):
    line_string = LineString([Point(lon1, lat1), Point(lon2, lat2)])
    geod = Geod(ellps="WGS84")
    return geod.geometry_length(line_string)

# X['dist_to_center'] = X.apply(lambda row: np.sqrt((row.lat - row.lat_center) ** 2 + (row.lon - row.lon_center) ** 2), axis=1)
X['dist_to_center'] = X.apply(lambda row: meter_distance(row.lat, row.lon, row.lat_center, row.lon_center), axis=1)
X[20:30]






C:\Users\oskar\AppData\Local\Temp/ipykernel_27920/240993591.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  X = X.merge(X.groupby(['municipality_name'])['lat', 'lon', 'revenue'].apply(lambda x: x.sum() / (x.count()))[['lat', 'lon']], on='municipality_name', suffixes=(None, '_center'))


,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,municipality_name,lat_center,lon_center,dist_to_center
20,815533852-915572677-772333,2016,DRAMMEN RESTAURANTDRIFT,1.1.6.3,Restaurants and cafes,6020303,AMTMAND BLOMS GATE 2,59.743899,10.206327,NaN,NaN,11.376,Drammen,59.740315,10.194234,788.617913
21,815533852-915572677-772333,2016,DRAMMEN RESTAURANTDRIFT,1.1.6.3,Restaurants and cafes,6020303,AMTMAND BLOMS GATE 2,59.743899,10.206327,NaN,NaN,11.376,Drammen,59.740315,10.194234,788.617913
22,978677975-971903880-18517,2016,BØRSEN FESTLOKALER OG CATERING AS,1.1.6.3,Restaurants and cafes,6020303,BRAGERNES TORG 13,59.743104,10.204928,NaN,NaN,5.121,Drammen,59.740315,10.194234,676.914090
23,978677975-971903880-18517,2016,BØRSEN FESTLOKALER OG CATERING AS,1.1.6.3,Restaurants and cafes,6020303,BRAGERNES TORG 13,59.743104,10.204928,NaN,NaN,5.121,Drammen,59.740315,10.194234,676.914090
24,814458172-914549639-729505,2016,BUTIKKDRIFT DRAMMEN AS,1.1.6.4,Belongs to dining,6020303,NEDRE STORGATE 6,59.742854,10.206751,NaN,Magasinet Drammen,0.000,Drammen,59.740315,10.194234,758.629774
25,814458172-914549639-729505,2016,BUTIKKDRIFT DRAMMEN AS,1.1.6.4,Belongs to dining,6020303,NEDRE STORGATE 6,59.742854,10.206751,NaN,Magasinet Drammen,0.000,Drammen,59.740315,10.194234,758.629774
26,987558377-882230562-38392,2016,SNAPPYS BRAGERNES AS,1.1.9.0,Fast food,6020303,NEDRE STORGATE 23,59.742875,10.210364,NaN,NaN,7.397,Drammen,59.740315,10.194234,950.842252
27,987558377-882230562-38392,2016,SNAPPYS BRAGERNES AS,1.1.9.0,Fast food,6020303,NEDRE STORGATE 23,59.742875,10.210364,NaN,NaN,7.397,Drammen,59.740315,10.194234,950.842252
28,974634651-981922174-46289,2016,PIGEN CAFE & BAR,1.2.3.0,Coffee shops,6020303,BRAGERNES TORG 8,59.744258,10.205450,NaN,NaN,18.438,Drammen,59.740315,10.194234,768.616938
29,974634651-981922174-46289,2016,PIGEN CAFE & BAR,1.2.3.0,Coffee shops,6020303,BRAGERNES TORG 8,59.744258,10.205450,NaN,NaN,18.438,Drammen,59.740315,10.194234,768.616938


In [180]:
X[110:130]

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,municipality_name,lat_center,lon_center,dist_to_center
110,959146578-979518706-49399,2016,FROST GULSKOGEN SENTER,2.4.6.0,Clothing stores,6020509,GULDLISTEN 35,59.742529,10.157972,NaN,Gulskogen Senter,7.246,Drammen,59.740315,10.194234,2054.085593
111,959146578-979518706-49399,2016,FROST GULSKOGEN SENTER,2.4.6.0,Clothing stores,6020509,GULDLISTEN 35,59.742529,10.157972,NaN,Gulskogen Senter,7.246,Drammen,59.740315,10.194234,2054.085593
112,914859875-914905605-745984,2016,I-E MOTE GULSKOGEN,2.4.6.0,Clothing stores,6020509,GULDLISTEN 35,59.742529,10.157972,NaN,Gulskogen Senter,4.834,Drammen,59.740315,10.194234,2054.085593
113,914859875-914905605-745984,2016,I-E MOTE GULSKOGEN,2.4.6.0,Clothing stores,6020509,GULDLISTEN 35,59.742529,10.157972,NaN,Gulskogen Senter,4.834,Drammen,59.740315,10.194234,2054.085593
114,995039826-995062089-6950,2016,KOZMOS GULSKOGEN SENTER,2.6.2.0,Toy stores,6020509,GULDLISTEN 35,59.742529,10.157972,SCANDITOY,Gulskogen Senter,6.293,Drammen,59.740315,10.194234,2054.085593
115,995039826-995062089-6950,2016,KOZMOS GULSKOGEN SENTER,2.6.2.0,Toy stores,6020509,GULDLISTEN 35,59.742529,10.157972,SCANDITOY,Gulskogen Senter,6.293,Drammen,59.740315,10.194234,2054.085593
116,917387923-986206469-55511,2016,EROTICA,2.7.4.0,Erotica shops,6020509,GULDLISTEN 35,59.742529,10.157972,NaN,Gulskogen Senter,2.009,Drammen,59.740315,10.194234,2054.085593
117,917387923-986206469-55511,2016,EROTICA,2.7.4.0,Erotica shops,6020509,GULDLISTEN 35,59.742529,10.157972,NaN,Gulskogen Senter,2.009,Drammen,59.740315,10.194234,2054.085593
118,997427173-971972815-8030,2016,BUDDY - ZOO HUSET GULSKOGEN,2.7.5.0,Pet stores,6020509,GULDLISTEN,59.743544,10.156422,ZOOKJEDEN BUDDY,Gulskogen Senter,10.089,Drammen,59.740315,10.194234,2156.560497
119,997427173-971972815-8030,2016,BUDDY - ZOO HUSET GULSKOGEN,2.7.5.0,Pet stores,6020509,GULDLISTEN,59.743544,10.156422,ZOOKJEDEN BUDDY,Gulskogen Senter,10.089,Drammen,59.740315,10.194234,2156.560497
